## Extract articles from website links

Aanpassing vanwege extra link titels





In [13]:
# book_link = 'https://www.ejustice.just.fgov.be/eli/wet/1804/03/21/1804032150/justel'
# book_name = 'oud_1_personen'
# book_link ='http://www.ejustice.just.fgov.be/eli/wet/1804/03/21/1804032151/justel'
# book_name= 'oud_2_goederen'
# book_link = 'http://www.ejustice.just.fgov.be/eli/wet/1804/03/21/1804032152/justel'
# book_name = 'oud_3_eigendomsverkrijging_1'
# book_link = 'http://www.ejustice.just.fgov.be/eli/wet/1804/03/21/1804032153/justel'
# book_name = 'oud_3_eigendomsverkrijging_2'
# book_link = 'http://www.ejustice.just.fgov.be/eli/wet/1804/03/21/1804032154/justel'
# book_name = 'oud_3_eigendomsverkrijging_3'
# book_link = 'http://www.ejustice.just.fgov.be/eli/wet/1804/03/21/1804032155/justel'
# book_name = 'oud_3_eigendomsverkrijging_4'
book_link = 'http://www.ejustice.just.fgov.be/eli/wet/2004/07/16/2004009511/justel'
book_name = 'privaatrecht'
wetboek = 'Wetboek van internationaal privaatrecht'
boekdeel=""


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [3]:
res = requests.get(book_link)
soup = BeautifulSoup(res.text, 'html.parser')

In [4]:
# with open('code_civil.txt', 'w') as file:
#     file.write(res.text)

In [4]:
# Find the <a> element with name="texte"
a_element = soup.find('a', attrs={'name': 'texte'})

# Find the next <table> element after the <a> element
table = a_element.find_next('table')
tr_elements = table.find_all('tr')
if len(tr_elements) >= 2:
    html_text = tr_elements[1]
else:
    print("There are less than two <tr> elements.")

html_text

<tr>
<th align="left" colspan="3" width="100%">
<a href="#LNKR0001" name="LNK0001">HOOFDSTUK I.</a> - Algemene bepalingen.<br/><br/>  <a href="#LNKR0002" name="LNK0002">Afdeling 1.</a> - Voorafgaande bepaling.<br/><br/>  <a href="#LNKR0003" name="LNK0003">Bedoelde aangelegenheid.</a><br/><br/>  <a name="Art.1">Artikel </a> <a href="#LNK0004"> 1</a>. Deze wet regelt een aangelegenheid als bedoeld in artikel 78 van de Grondwet, met uitzondering van de artikelen 5 tot 14, 23, §§ 1 en 2, 27, § 1, vierde lid en § 2, 31, § 1, derde lid, 32, 33, 36, 40, 42, 43, 59, 61, 66, 73, 77, 85, 86, 96, 97, 109, 118, 121, § 4, 123, 126, § 1, 134, 135, 136 en 139, 5° en 8°, die een aangelegenheid regelen als bedoeld in artikel 77 van de Grondwet.<br/><br/>  <a href="#LNKR0004" name="LNK0004">Afdeling 2.</a> - Onderwerp.<br/><br/>  <a href="#LNKR0005" name="LNK0005">Onderwerp.</a><br/><br/>  <a href="#Art.1" name="Art.2">Art.</a> <a href="#LNK0006"> 2</a>. Onder voorbehoud van de toepassing van internatio

In [5]:

# Remove all <sup> elements
for sup in html_text.find_all('sup'):
    sup.decompose()

In [6]:
# import copy
# html = copy.deepcopy(html_text)
html=html_text

In [7]:
for a in html.find_all('a'):
    tekst = a.string
    if tekst is None:
        print("NONE " + str(a))
        continue
    elif any(keyword in tekst for keyword in ['BOEK', 'Boek', 'TITEL', 'Titel', 'HOOFDSTUK', 'Hoofdstuk', 'AFDELING', 'Afdeling', 'Onderafdeling']):
        a.string = '#' + tekst + '#'
    elif a.get('name') and "Art." in a['name']:
        a.string = '#' + a['name'] + '# ' + tekst
        # print("NEW ARTICLE" + a['name'])
    else:
        a.string=''
# html

NONE <a href="#LNKR0098" name="LNK0098">[ Recht toepasselijk op de onderhoudsverplichting die niet voortvloeit uit familiebetrekkingen]</a>


In [8]:
articles_txt = html.get_text()

In [9]:
with open('articles.txt', 'w') as file:
    file.write(articles_txt)

In [10]:

# Remove all text that starts with "  ---------" and ends with ">"
cleaned_txt = re.sub('---------.*?#', '#', str(articles_txt))

#Remove [ ... ]
cleaned_txt = re.sub('\[\s*\.*\s*\]', '', str(cleaned_txt))

#Remove [(oud artikel ...)]
cleaned_txt = re.sub('\(\s*oud artikel[^\)]*\)', '', str(cleaned_txt))

#Remove all [ and ]
cleaned_txt = re.sub('\[', '', str(cleaned_txt))
cleaned_txt = re.sub('\]', '', str(cleaned_txt))

#Remove all punctuation at the end of a title
cleaned_txt = re.sub('\.#', '#', str(cleaned_txt))

#Remove everything between <> except when "Opgeheven" is in between them
cleaned_txt = re.sub('<(?!Opgeheven|opgeheven)[^>]*>', ' ', str(cleaned_txt))

#Remove the text when there is a space after Art. 
cleaned_txt = re.sub('Art\.\s+\S*\.', '', str(cleaned_txt))

#Remove all unnecessary whitespace
cleaned_txt = re.sub('\s+', ' ', str(cleaned_txt))

print(cleaned_txt)

 #HOOFDSTUK I# - Algemene bepalingen. #Afdeling 1# - Voorafgaande bepaling. #Art.1# Artikel . Deze wet regelt een aangelegenheid als bedoeld in artikel 78 van de Grondwet, met uitzondering van de artikelen 5 tot 14, 23, §§ 1 en 2, 27, § 1, vierde lid en § 2, 31, § 1, derde lid, 32, 33, 36, 40, 42, 43, 59, 61, 66, 73, 77, 85, 86, 96, 97, 109, 118, 121, § 4, 123, 126, § 1, 134, 135, 136 en 139, 5° en 8°, die een aangelegenheid regelen als bedoeld in artikel 77 van de Grondwet. #Afdeling 2# - Onderwerp. #Art.2# Onder voorbehoud van de toepassing van internationale verdragen, van het recht van de Europese Unie of van bepalingen in bijzondere wetten, regelt deze wet voor internationale gevallen de bevoegdheid van de Belgische rechters, de aanwijzing van het toepasselijk recht en de voorwaarden voor de uitwerking in België van buitenlandse rechterlijke beslissingen en authentieke akten in burgerlijke zaken en in handelszaken. #Afdeling 3# - Vaststelling van de nationaliteit, de woonplaats en

In [11]:
with open('cleaned.txt', 'w') as file:
    file.write(cleaned_txt)

# Create table


In [14]:
#Create 2 columns: "Titel, text"
articles_lst = []
verwijderd = []
code= wetboek
boek=boekdeel
titel=""
hoofdstuk=""
afdeling=""
onderafdeling=""

parts = cleaned_txt.split("#")

def set_titles(b="", t="", h="", a="", o=""):
    global boek, titel, hoofdstuk, afdeling, onderafdeling
    boek=b
    titel=t
    hoofdstuk=h
    afdeling=a
    onderafdeling=o

for i in range(1, len(parts), 2):
    name = parts[i].strip()
    text = parts[i+1].strip()
    if (("<Opgeheven bij W " in text) or ("(Opgeheven)" in text) or ("(opgeheven)" in text)) and not re.search(r'§.*?opgeheven', text):
        verwijderd.append([name, text])
        continue
    if "TOEKOMSTIG RECHT" in name:
        #Remove the TOEKOMSTIG RECHT parts
        continue
    if "Art." in name:
        articles_lst.append([name, text, code+", "+boek+", "+titel+", "+hoofdstuk+", "+afdeling+", "+onderafdeling]) 
    else:
        text = re.sub('^\s*-\s*', '', text)
        text = re.sub('\s*\.$', '', text)
        articles_lst.append([name, text, ""])
        if "boek" in name.lower():
            set_titles(name)
        elif "titel" in name.lower():
            set_titles(boek ,name)
        elif "hoofdstuk" in name.lower():
            set_titles(boek, titel, name)
        elif "Onderafdeling" in name:
            set_titles(boek, titel, hoofdstuk, afdeling, name)
        elif "afdeling" in name.lower():
            set_titles(boek, titel, hoofdstuk, name)


len(verwijderd), verwijderd

(7,
 [['Art.79',
   '<Opgeheven bij W , art. 57, 010; Inwerkingtreding : 17-08-2015>'],
  ['Art.80',
   '<Opgeheven bij W , art. 57, 010; Inwerkingtreding : 17-08-2015>'],
  ['Art.81',
   '<Opgeheven bij W , art. 57, 010; Inwerkingtreding : 17-08-2015>'],
  ['Art.82',
   '<Opgeheven bij W , art. 57, 010; Inwerkingtreding : 17-08-2015>'],
  ['Art.83',
   '<Opgeheven bij W , art. 57, 010; Inwerkingtreding : 17-08-2015>'],
  ['Art.84',
   '<Opgeheven bij W , art. 57, 010; Inwerkingtreding : 17-08-2015>'],
  ['Art.120',
   '<Opgeheven bij W , art. 64, 011; Inwerkingtreding : 01-05-2018>']])

In [15]:
columns= ["title", "name", "section"]
df_parts = pd.DataFrame(articles_lst, columns=columns)
df_parts

title                                               name  \
0    HOOFDSTUK I                                Algemene bepalingen   
1     Afdeling 1                              Voorafgaande bepaling   
2          Art.1  Artikel . Deze wet regelt een aangelegenheid a...   
3     Afdeling 2                                          Onderwerp   
4          Art.2  Onder voorbehoud van de toepassing van interna...   
..           ...                                                ...   
172      Art.138  In artikel 3 van de wet van 8 augustus 1997 in...   
173   Afdeling 3                               Opheffingsbepalingen   
174      Art.139  Volgende bepalingen worden opgeheven : 1° de a...   
175   Afdeling 4                                   Inwerkingtreding   
176      Art.140  Deze wet treedt in werking op de eerste dag va...   

                                               section  
0                                                       
1                                                       
2    Wetboek van internationaal privaatrecht, , , H...  
3                                                       
4    Wetboek van internationaal privaatrecht, , , H...  
..                                                 ...  
172  Wetboek van internationaal privaatrecht, , , H...  
173                                                     
174  Wetboek van internationaal privaatrecht, , , H...  
175                                                     
176  Wetboek van internationaal privaatrecht, , , H...  

[177 rows x 3 columns]

In [16]:
df_sections = df_parts[df_parts["section"].str.strip().eq('')]
df_articles = df_parts[~df_parts["section"].str.strip().eq('')]
len(df_parts), len(df_sections), len(df_articles)

(177, 39, 138)

In [17]:
df_articles.to_csv('csv_scraped/'+ book_name + '_articles.csv', index=False)
df_sections.to_csv('csv_scraped/'+ book_name + '_sections.csv', index=False)